# 手動最適化

In [ ]:
import numpy as np
from convinience import Spectrum, Band, synthesisSpectrum
from convinience import gaussian_profile, compensation, calc_totalPower ,attenuation
from mso58trans import MSO58Wrapper, MSO58transData
from n9010btrans import N9010BtransData
from shutter import servo_shutter
from WSMethods import uploadProfile, getWSrange
from sklearn.decomposition import PCA
from csv import DictWriter
import datetime
import copy
from time import sleep
import os
import ipywidgets as widgets
from IPython.display import display
from matplotlib import pyplot as plt
from ipyfilechooser import FileChooser
from bayes_opt import BayesianOptimization as _BayesianOptimization
from bayes_opt.logger import JSONLogger as _JSONLogger
from bayes_opt.event import Events
from scipy import interpolate
import pyvisa

In [ ]:
class BayesianOptimization(_BayesianOptimization):
    @property
    def meta_data(self,):
        try: return self._space.meta_data
        except AttributeError: return []

    def register(self,*args,**kwargs):
        try: meta_data = kwargs.pop('meta_data')
        except KeyError: meta_data = None
        try: self._space.meta_data.append(meta_data)
        except AttributeError:
            self._space.meta_data = [meta_data]
        super().register(*args,**kwargs)

class JSONLogger(_JSONLogger):
    def update(self, event, instance):
        import json
        if event == Events.OPTIMIZATION_STEP:
            data = dict(instance.res[-1])
            
            data["meta_data"] = instance.meta_data[-1]

            now, time_elapsed, time_delta = self._time_metrics()
            data["datetime"] = {
                "datetime": now,
                "elapsed": time_elapsed,
                "delta": time_delta,
            }
            

            if "allowed" in data: # fix: github.com/fmfn/BayesianOptimization/issues/361
                data["allowed"] = bool(data["allowed"])

            with open(self._path, "a") as f:
                f.write(json.dumps(data) + "\n")

        self._update_tracker(event, instance)

def load_logs(optimizer, logs):
    """Load previous ...

    """
    import json

    if isinstance(logs, str):
        logs = [logs]

    for log in logs:
        with open(log, "r") as j:
            while True:
                try:
                    iteration = next(j)
                except StopIteration:
                    break

                iteration = json.loads(iteration)
                try:
                    optimizer.register(
                        params=iteration["params"],
                        target=iteration["target"],
                        meta_data=iteration.get("meta_data")
                    )
                except KeyError:
                    pass

    return optimizer

In [ ]:
# 各種設定周り ここにすべてまとめる

class Setting:
    def __init__(self):
        global hd_idx
        self.ws_ip                          = '169.254.6.8'
        self.ws_Freq_resolution             = 0.001 # 0.001THz = 1GHz
        self.ws_Freqrange                   = None
        self.optSpectrum : Spectrum         = None
        self.mso58_address                  = 'USB::0x0699::0x0530::C043144::INSTR'
        self.homodyne_port                  = 1
        self.fastFramePulseNum              = 5
        #self.quadrature_method              = lambda volts: PCA().fit_transform(volts.reshape((-1,volts.shape[-1])))[:,0].reshape(volts.shape[:-1])
        self.quadrature_method              = lambda volts: volts.reshape(-1,volts.shape[-1])[:,hd_idx].reshape(volts.shape[:-1])
        self.n9010b_address                 = 'TCPIP0::K-N9010B-42308.local::hislip0::INSTR'
        self.hd_trace                       = 1
        self.speana_idx_from                = 600
        self.speana_idx_to                  = 1700
        self.targetLOPower                  = 1
        self.shutter_port                   = 'COM7'
        self.pumpShutterPort : int          = 2
        self.param_csv                      = f"{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}record.csv"


    def condition_approval(self):
        assert self.ws_Freqrange             is not None
        assert self.optSpectrum              is not None
        assert self.optSpectrum.powerdensity is not None
        assert self.optSpectrum.frequency    is not None
        assert self.optSpectrum.wavelength   is not None
        assert self.optSpectrum.phase        is not None
        assert self.fastFramePulseNum        is not None
        assert self.quadrature_method        is not None
        assert self.targetLOPower            is not None
        assert self.shutter_port             is not None
        assert self.pumpShutterPort          is not None
        assert self.param_csv                is not None


setting = Setting()

startFreq, endFreq    = 191.05, 196.475# getWSrange(setting.ws_ip)
setting.ws_Freqrange  = np.arange(startFreq, endFreq, setting.ws_Freq_resolution)

setting.optSpectrum                = Spectrum()
setting.optSpectrum.band           = Band()
setting.optSpectrum.frequency      = np.load('optFreq.npy')
setting.optSpectrum.powerdensity   = np.load('optSpectrum.npy')
setting.optSpectrum.phase          = np.zeros_like(setting.optSpectrum.frequency)

setting.condition_approval()

### 最適化フェーズ

In [ ]:
class State:
    def __init__(self):
        self.current_spectrum = None
        self.former_spectrum  = None
        self.former_diff = 0
        self.diff_history = []

state = State()

In [ ]:

def quad(w, pulseNum, quadrature_func):
    frameLength = w.shape[1]
    frames      = w.shape[0]
    period = [int(i * frameLength/pulseNum) for i in range(pulseNum)]
    duration = int(frameLength/pulseNum)
    quadratures = np.zeros((frames, pulseNum))

    volts = np.zeros((frames,pulseNum,duration))

    for i in range(pulseNum):
        s = period[i]
        volts[:,i,:] = w[:,s:(s+duration)]

    quadratures = quadrature_func(volts)
    return quadratures


def evaluation(quadratures):
    return quadratures.var()



def param_gen(state_arg: State = None):
    global state
    if state_arg is None: state_arg = state
    band = Band()
    band.frequency          = setting.ws_Freqrange
    filterSpectrum          = state_arg.current_spectrum

    po_center_wavelength = np.random.uniform(1544, 1546)
    po_height = 0.5*np.random.randn()
    po_bandwidth = 0.5

    ph_center_wavelength = np.random.uniform(1544, 1546)
    ph_height = 1*np.random.randn()
    ph_bandwidth = 0.5

    spectrum_diff = Spectrum()
    spectrum_diff.band = copy.deepcopy(band)
    spectrum_diff.powerdensity = po_height * np.exp(-((band.wavelength- po_center_wavelength)/(po_bandwidth/2))**2)
    spectrum_diff.phase = ph_height * np.exp(-((band.wavelength- ph_center_wavelength)/(ph_bandwidth/2))**2)

    filterSpectrum = synthesisSpectrum(filterSpectrum, spectrum_diff, band)

    
    state_arg.former_spectrum = state_arg.current_spectrum
    state_arg.current_spectrum = filterSpectrum
    return filterSpectrum


def param_gen_yield():
    while True:
        yield param_gen()

def param_unwind(state_arg: State = None):
    global state
    if state_arg is None: state_arg = state
    state_arg.current_spectrum = state_arg.former_spectrum
    state_arg.former_spectrum = None

def set_param(state_arg: State = None):
    global state
    if state_arg is None: state_arg = state
    filterSpectrum = state_arg.current_spectrum
    band = Band()
    band.frequency          = setting.ws_Freqrange
    compensated_Spectrum    = compensation(
            setting.optSpectrum,
            targetSpectrum      =filterSpectrum,
            band                =band
        )
    #compensated_Spectrum = attenuation(compensated_Spectrum, total_power - setting.targetLOPower)
    filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
    r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

def open_pump_shutter():
    global state
    try: servo_shutter(port=setting.shutter_port).open([setting.pumpShutterPort])
    except Exception: pass

def close_pump_shutter():
    global stete
    try: servo_shutter(port=setting.shutter_port).close([setting.pumpShutterPort])
    except Exception: pass

def set_lo_EOM_high_voltage():
    global lo_EOM_high_voltage
    rm = pyvisa.ResourceManager()
    with rm.open_resource("TCPIP0::A-33600-00000.local::inst0::INSTR") as instr:
        command = f"SOURce2:VOLTage:HIGH {lo_EOM_high_voltage}"
        print('>'+command)
        print(instr.write(command))

def measure_diff_1push():
    ## pump off
    servo_shutter(port=setting.shutter_port).close([setting.pumpShutterPort])
    sleep(1)
    ## osc_com
    osc_handler = MSO58Wrapper(setting.mso58_address)
    osc_handler.push_single()
    t, w = None, None
    while True:
        try:
            t, w = osc_handler.transfer2byte(setting.homodyne_port)
            break
        except:
            sleep(1)
            continue
    ## quad
    quadratures = quad(w, setting.fastFramePulseNum, setting.quadrature_method)
    ## eval_shotnoise
    shot_var = quadratures.var()
    flatten_quadratures = quadratures.flatten()
    plt.scatter(range(quadratures.shape[0]),flatten_quadratures)
    plt.show()

    ## pump on
    servo_shutter(port=setting.shutter_port).open([setting.pumpShutterPort])
    sleep(1)
    ## osc_com
    osc_handler = MSO58Wrapper(setting.mso58_address)
    osc_handler.push_single()
    t, w = None, None
    while True:
        try:
            t, w = osc_handler.transfer2byte(setting.homodyne_port)
            break
        except:
            sleep(1)
            continue
    ## quad
    quadratures = quad(w, setting.fastFramePulseNum, setting.quadrature_method)
    ## eval_squeeze
    pumped_var = quadratures.var()
    flatten_quadratures = quadratures.flatten()
    plt.scatter(range(quadratures.shape[0]),flatten_quadratures)
    plt.show()
    print(10*np.log10(pumped_var) - 10*np.log10(shot_var))

def measure_diff_1push_speana():

    idx_from = setting.speana_idx_from
    idx_to   = setting.speana_idx_to

    ## pump off
    servo_shutter(port=setting.shutter_port).close([setting.pumpShutterPort])
    sleep(1)
    ## speana_com
    speana_handler = N9010BtransData(setting.n9010b_address)
    trace_shot = speana_handler.transfer(setting.hd_trace)
    shot_level = trace_shot[idx_from:idx_to].mean()

    
    ## pump on
    servo_shutter(port=setting.shutter_port).open([setting.pumpShutterPort])
    sleep(1)
    ## speana_com
    trace_pumped = speana_handler.transfer(setting.hd_trace)
    pumped_level = trace_pumped[idx_from: idx_to].mean()
    plt.plot(trace_shot)
    plt.plot(trace_pumped)
    plt.show()
    print(pumped_level - shot_level)

def measure_level_1push_speana():
    
    idx_from = setting.speana_idx_from
    idx_to   = setting.speana_idx_to

    ## speana_com
    speana_handler = N9010BtransData(setting.n9010b_address)
    trace_shot = speana_handler.transfer(setting.hd_trace)
    shot_level = trace_shot[idx_from:idx_to].mean()
    plt.plot(trace_shot)
    plt.show()
    print(shot_level)

def init_bayesian01():
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-2":(-5,5),"-1":(-5,5),"0":(-5,5),"1":(-5,5),"2":(-5,5)},
        verbose=2,
        random_state=1
    )
    from bayes_opt.logger import JSONLogger
    from bayes_opt.event import Events
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian01.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
    


def loop_phase_bayesian01():
    # 中心 1545.3 波長幅 0.5nm {-2,-1,0,1,2}*0.5 + 1545.3nmの波長を制御します。 
    # 境界なめらか処理はなしです。まあなくてもいいんじゃない？
    global state
    global bayesian_optimizer
    while True:
        idx_from = setting.speana_idx_from
        idx_to   = setting.speana_idx_to
        # 
        from bayes_opt import UtilityFunction
        utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
        next_probe_point = bayesian_optimizer.suggest(utility)
        next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
#        next_probe_point[-3] = 0 # 境界なめらか処理
#        next_probe_point[3] = 0 # 境界なめらか処理
        # phase generation(next_probe_point)
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                    centerWavelength    =1545.3,
                    bandWidth           =0.5, band=band,
                    dispersion          =-0.9
                )  
        x = list(next_probe_point_trans.keys())
        x.sort()
        y = [next_probe_point_trans[i] for i in x]
        x = np.array([float(i) for i in x]) * 0.5 + 1545.3
        akima = interpolate.Akima1DInterpolator(x,y)
        filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
        state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
        compensated_Spectrum = copy.deepcopy(filterSpectrum)
        
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        sleep(10)

        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        shot_level = trace_shot[idx_from:idx_to].mean()
        bayesian_optimizer.register(
            params=next_probe_point,
            target= -(shot_level - (-60)) # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。 
        )

def init_bayesian02():
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-1":(-5,5),"-0.5":(-5,5),"0":(-5,5),"0.5":(-5,5),"1":(-5,5)},
        verbose=2,
        random_state=1
    )
    from bayes_opt.logger import JSONLogger
    from bayes_opt.event import Events
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian02.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

def loop_phase_bayesian02():
    # 中心 1545.3 波長幅 0.5nm {-1,-0.5,0,0.5,1}*0.5 + 1545.3nmの波長を制御します。 
    # 境界なめらか処理はありません。なくてもいいやろ。必要性を感じない。
    global state
    global bayesian_optimizer
    while True:
        idx_from = setting.speana_idx_from
        idx_to   = setting.speana_idx_to
        # 
        from bayes_opt import UtilityFunction
        utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
        next_probe_point = bayesian_optimizer.suggest(utility)
        next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
#        next_probe_point[-1.5] = 0 # 境界なめらか処理
#        next_probe_point[1.5] = 0 # 境界なめらか処理
        # phase generation(next_probe_point)
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                    centerWavelength    =1545.3,
                    bandWidth           =0.5, band=band,
                    dispersion          =-0.9
                )  
        x = list(next_probe_point_trans.keys())
        x.sort()
        y = [next_probe_point_trans[i] for i in x]
        x = np.array([float(i) for i in x]) * 0.5 + 1545.3
        akima = interpolate.Akima1DInterpolator(x,y)
        filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
        state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
        compensated_Spectrum = copy.deepcopy(filterSpectrum)
        
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        sleep(10)

        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        shot_level = trace_shot[idx_from:idx_to].mean()
        bayesian_optimizer.register(
            params=next_probe_point,
            target= -(shot_level - (-60)) # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。 
        )

def init_bayesian03():
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-2":(-5,5),"-1":(-5,5),"0":(-5,5),"1":(-5,5),"2":(-5,5)},
        verbose=2,
        random_state=1
    )
    from bayes_opt.logger import JSONLogger
    from bayes_opt.event import Events
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian03.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

def loop_phase_bayesian03():
    # これはシャッター制御ありの bayesian01です。
    # 中心 1545.3 波長幅 0.5nm {-2,-1,0,1,2}*0.5 + 1545.3nmの波長を制御します。 
    # 境界なめらか処理はなしです。まあなくてもいいんじゃない？
    global state
    global bayesian_optimizer
    while True:
        idx_from = setting.speana_idx_from
        idx_to   = setting.speana_idx_to
        # 
        from bayes_opt import UtilityFunction
        utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
        next_probe_point = bayesian_optimizer.suggest(utility)
        next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
#        next_probe_point[-3] = 0 # 境界なめらか処理
#        next_probe_point[3] = 0 # 境界なめらか処理
        # phase generation(next_probe_point)
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                    centerWavelength    =1545.3,
                    bandWidth           =0.5, band=band,
                    dispersion          =-0.9
                )  
        x = list(next_probe_point_trans.keys())
        x.sort()
        y = [next_probe_point_trans[i] for i in x]
        x = np.array([float(i) for i in x]) * 0.5 + 1545.3
        akima = interpolate.Akima1DInterpolator(x,y)
        filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
        state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
        compensated_Spectrum = copy.deepcopy(filterSpectrum)
        
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        close_pump_shutter()
        sleep(10)

        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        shot_level = trace_shot[idx_from:idx_to].mean()

        open_pump_shutter()
        sleep(5)
        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        sq_level = trace_shot[idx_from:idx_to].mean()

        bayesian_optimizer.register(
            params=next_probe_point,
            target= -(sq_level - shot_level) # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。 
        )

def init_bayesian04():
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-2":(-5,5),"-1":(-5,5),"0":(-5,5),"1":(-5,5),"2":(-5,5)},
        verbose=2,
        random_state=1
    )
    from bayes_opt.logger import JSONLogger
    from bayes_opt.event import Events
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian04.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

def loop_phase_bayesian04():
    # 中心 1545.344 波長幅 0.5nm {-2,-1,0,1,2}*0.5 + 1545.3nmの波長を制御します。 
    # 境界なめらか処理はなしです。まあなくてもいいんじゃない？
    # bayesian01を中心波長1545.344にしたつもりでしたが、
    # パワー中心1545.344 位相中心1545.344でしたね。
    # まあパワーのプロファイルの中心がちゃんと1545.3になってるからいいですけど。
    global state
    global bayesian_optimizer
    while True:
        idx_from = setting.speana_idx_from
        idx_to   = setting.speana_idx_to
        # 
        from bayes_opt import UtilityFunction
        utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
        next_probe_point = bayesian_optimizer.suggest(utility)
        next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
#        next_probe_point[-3] = 0 # 境界なめらか処理
#        next_probe_point[3] = 0 # 境界なめらか処理
        # phase generation(next_probe_point)
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                    centerWavelength    =1545.344,
                    bandWidth           =0.5, band=band,
                    dispersion          =-0.9
                )  
        x = list(next_probe_point_trans.keys())
        x.sort()
        y = [next_probe_point_trans[i] for i in x]
        x = np.array([float(i) for i in x]) * 0.5 + 1545.3
        akima = interpolate.Akima1DInterpolator(x,y)
        filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
        state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
        compensated_Spectrum = copy.deepcopy(filterSpectrum)
        
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        sleep(10)

        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        shot_level = trace_shot[idx_from:idx_to].mean()
        bayesian_optimizer.register(
            params=next_probe_point,
            target= -(shot_level - (-60)) # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。 
        )

def init_bayesian05():
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-2":(-5,5),"-1":(-5,5),"0":(-5,5),"1":(-5,5),"2":(-5,5)},
        verbose=2,
        random_state=1
    )
    from bayes_opt.logger import JSONLogger
    from bayes_opt.event import Events
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian05.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

def loop_phase_bayesian05():
    # 中心 1545.344 波長幅 0.5nm {-2,-1,0,1,2}*0.5 + 1545.3nmの波長を制御します。 
    # 境界なめらか処理はなしです。まあなくてもいいんじゃない？
    # bayesian01を中心波長1545.344にしたつもりでしたが、
    # パワー中心1545.344 位相中心1545.344でしたね。
    # まあパワーのプロファイルの中心がちゃんと1545.3になってるからいいですけど。
    global state
    global bayesian_optimizer
    while True:
        idx_from = setting.speana_idx_from
        idx_to   = setting.speana_idx_to
        # 
        from bayes_opt import UtilityFunction
        utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
        next_probe_point = bayesian_optimizer.suggest(utility)
        next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
#        next_probe_point[-3] = 0 # 境界なめらか処理
#        next_probe_point[3] = 0 # 境界なめらか処理
        # phase generation(next_probe_point)
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                    centerWavelength    =1545.483,
                    bandWidth           =0.5, band=band,
                    dispersion          =-0.9
                )  
        x = list(next_probe_point_trans.keys())
        x.sort()
        y = [next_probe_point_trans[i] for i in x]
        x = np.array([float(i) for i in x]) * 0.5 + 1545.3
        akima = interpolate.Akima1DInterpolator(x,y)
        filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
        state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
        compensated_Spectrum = copy.deepcopy(filterSpectrum)
        
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        sleep(10)

        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        shot_level = trace_shot[idx_from:idx_to].mean()
        bayesian_optimizer.register(
            params=next_probe_point,
            target= -(shot_level - (-60)), # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。 
        )

def init_bayesian06():
    # ついに、LOの角度制御も行います。
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-2":(-5,5),"-1":(-5,5),"0":(-5,5),"1":(-5,5),"2":(-5,5),"EOM_rad":(0,np.pi/2)},
        verbose=2,
        random_state=1
    )
    from bayes_opt.logger import JSONLogger
    from bayes_opt.event import Events
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian06.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)


def loop_phase_bayesian06():
    # bayesian05派生 EOM角度最適化あり。ただしベイズの枠組みにおいて。
    global state
    global bayesian_optimizer
    while True:
        idx_from = setting.speana_idx_from
        idx_to   = setting.speana_idx_to
        # 
        from bayes_opt import UtilityFunction
        utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
        next_probe_point = bayesian_optimizer.suggest(utility)
        next_probe_point_temp = next_probe_point.copy()
        EOM_rad = next_probe_point_temp.pop('EOM_rad')
        next_probe_point_trans = {float(k):v for k, v in next_probe_point_temp.items()}
#        next_probe_point[-3] = 0 # 境界なめらか処理
#        next_probe_point[3] = 0 # 境界なめらか処理
        # phase generation(next_probe_point)
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                    centerWavelength    =1545.483,
                    bandWidth           =0.5, band=band,
                    dispersion          =-0.9
                )  
        x = list(next_probe_point_trans.keys())
        x.sort()
        y = [next_probe_point_trans[i] for i in x]
        x = np.array([float(i) for i in x]) * 0.5 + 1545.3
        akima = interpolate.Akima1DInterpolator(x,y)
        filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
        state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
        compensated_Spectrum = copy.deepcopy(filterSpectrum)
        
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        global lo_EOM_high_voltage
        lo_EOM_high_voltage = 0.5+3.430*EOM_rad/np.pi
        set_lo_EOM_high_voltage()

        sleep(10)

        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        shot_level = trace_shot[idx_from:idx_to].mean()
        bayesian_optimizer.register(
            params=next_probe_point,
            target= -(shot_level - (-60)) # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。 
        )

def init_bayesian07():
    # ついに、LOの角度制御も行います。
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-2":(-5,5),"-1":(-5,5),"0":(-5,5),"1":(-5,5),"2":(-5,5)},
        verbose=2,
        random_state=1
    )
    from bayes_opt.logger import JSONLogger
    from bayes_opt.event import Events
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian07.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

def loop_phase_bayesian07():
    # もともとbayesian05ですが、それにEOM角度操作を加えました。
    global state
    global bayesian_optimizer
    while True:
        idx_from = setting.speana_idx_from
        idx_to   = setting.speana_idx_to
        # 
        from bayes_opt import UtilityFunction
        utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
        next_probe_point = bayesian_optimizer.suggest(utility)
        next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
#        next_probe_point[-3] = 0 # 境界なめらか処理
#        next_probe_point[3] = 0 # 境界なめらか処理
        # phase generation(next_probe_point)
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                    centerWavelength    =1545.483,
                    bandWidth           =0.5, band=band,
                    dispersion          =-0.9
                )  
        x = list(next_probe_point_trans.keys())
        x.sort()
        y = [next_probe_point_trans[i] for i in x]
        x = np.array([float(i) for i in x]) * 0.5 + 1545.3
        akima = interpolate.Akima1DInterpolator(x,y)
        filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
        state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
        compensated_Spectrum = copy.deepcopy(filterSpectrum)
        
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        sleep(5)

        shot_level_list = []
        for deg in [0,15,30,45,60,75,90]:
            global lo_EOM_high_voltage
            lo_EOM_high_voltage = 0.5+3.430*deg/180.0 # 0.5Vはちょっとしたオフセットです。
            set_lo_EOM_high_voltage()

            sleep(5)
            speana_handler = N9010BtransData(setting.n9010b_address)
            trace_shot = speana_handler.transfer(setting.hd_trace)
            shot_level = trace_shot[idx_from:idx_to].mean()
            shot_level_list.append(shot_level)
        bayesian_optimizer.register(
            params=next_probe_point,
            target= -(np.min(shot_level_list) - (-60)) # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。 
        )

def init_bayesian08():
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-2":(-5,5),"-1":(-5,5),"0":(-5,5),"1":(-5,5),"2":(-5,5)},
        verbose=2,
        random_state=1
    )
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian08.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

def loop_phase_bayesian08():
    # 中心 1545.344 波長幅 0.5nm {-2,-1,0,1,2}*0.5 + 1545.3nmの波長を制御します。 
    # 境界なめらか処理はなしです。まあなくてもいいんじゃない？
    # bayesian01を中心波長1545.344にしたつもりでしたが、
    # パワー中心1545.344 位相中心1545.344でしたね。
    # まあパワーのプロファイルの中心がちゃんと1545.3になってるからいいですけど。
    global state
    global bayesian_optimizer
    while True:
        idx_from = setting.speana_idx_from
        idx_to   = setting.speana_idx_to
        # 
        from bayes_opt import UtilityFunction
        utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
        next_probe_point = bayesian_optimizer.suggest(utility)
        next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
#        next_probe_point[-3] = 0 # 境界なめらか処理
#        next_probe_point[3] = 0 # 境界なめらか処理
        # phase generation(next_probe_point)
        band = Band()
        band.frequency          = setting.ws_Freqrange
        filterSpectrum          = gaussian_profile(
                    centerWavelength    =1545.483,
                    bandWidth           =0.5, band=band,
                    dispersion          =-0.9
                )  
        x = list(next_probe_point_trans.keys())
        x.sort()
        y = [next_probe_point_trans[i] for i in x]
        x = np.array([float(i) for i in x]) * 0.5 + 1545.3
        akima = interpolate.Akima1DInterpolator(x,y)
        filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
        state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
        compensated_Spectrum = copy.deepcopy(filterSpectrum)
        
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

        close_pump_shutter()
        sleep(10)

        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        shot_level = trace_shot[idx_from:idx_to].mean()

        open_pump_shutter()
        sleep(5)
        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        sq_level = trace_shot[idx_from:idx_to].mean()


        bayesian_optimizer.register(
            params=next_probe_point,
            target= -(sq_level - shot_level), # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。
            meta_data={'sq_level':sq_level, 'shot_level':shot_level}
        )

def init_bayesian09():
    global bayesian_optimizer
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"center_diff":(-0.5,0.5),"BW":(0.5,2),"-1.5":(-5,5),"-1":(-5,5),"0.5":(-5,5),"0":(-5,5),"0.5":(-5,5),"1":(-5,5),"1.5":(-5,5)},
        verbose=2,
        random_state=1,
        allow_duplicate_points=True
    )
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian09.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

def loop_phase_bayesian09():
    # 中心 1545.344 波長幅 0.5nm {-2,-1,0,1,2}*0.5 + 1545.3nmの波長を制御します。 
    # 境界なめらか処理はなしです。まあなくてもいいんじゃない？
    # bayesian01を中心波長1545.344にしたつもりでしたが、
    # パワー中心1545.344 位相中心1545.344でしたね。
    # まあパワーのプロファイルの中心がちゃんと1545.3になってるからいいですけど。
    global state
    global bayesian_optimizer
    while True:
        try:
            idx_from = setting.speana_idx_from
            idx_to   = setting.speana_idx_to
            # 
            from bayes_opt import UtilityFunction
            utility = UtilityFunction(kind="ucb", kappa=1, xi=0.0)
            next_probe_point = bayesian_optimizer.suggest(utility)
            next_probe_point_trans = copy.deepcopy(next_probe_point)
            center_diff = next_probe_point_trans.pop('center_diff')
            BW = next_probe_point_trans.pop('BW')
            next_probe_point_trans = {float(k):v for k, v in next_probe_point_trans.items()}
    #        next_probe_point[-3] = 0 # 境界なめらか処理
    #        next_probe_point[3] = 0 # 境界なめらか処理
            # phase generation(next_probe_point)
            band = Band()
            band.frequency          = setting.ws_Freqrange
            filterSpectrum          = gaussian_profile(
                        centerWavelength    =1545.3 + center_diff,
                        bandWidth           =BW, band=band,
                        dispersion          =-0.9
                    )  
            x = list(next_probe_point_trans.keys())
            x.sort()
            y = [next_probe_point_trans[i] for i in x]
            x = np.array([float(i) for i in x]) * 1 + 1545.3
            akima = interpolate.Akima1DInterpolator(x,y)
            filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
            state.current_spectrum = filterSpectrum # 補償はしていない(^_^;)
            compensated_Spectrum = copy.deepcopy(filterSpectrum)
            
            filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
            r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

            close_pump_shutter()
            sleep(10)

            speana_handler = N9010BtransData(setting.n9010b_address)
            trace_shot = speana_handler.transfer(setting.hd_trace)
            shot_level = trace_shot[idx_from:idx_to].mean()

            open_pump_shutter()
            sleep(5)
            speana_handler = N9010BtransData(setting.n9010b_address)
            trace_shot = speana_handler.transfer(setting.hd_trace)
            sq_level = trace_shot[idx_from:idx_to].mean()


            bayesian_optimizer.register(
                params=next_probe_point,
                target= -(sq_level - shot_level), # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。
                meta_data={'sq_level':sq_level, 'shot_level':shot_level}
            )
        except Exception: pass

def init_bayesian10():
    global bayesian_optimizer
    global state
    bayesian_optimizer = BayesianOptimization(
        f=None,
        pbounds={"-1.5":(-5,5),"-1":(-5,5),"-0.5":(-5,5),"0":(-5,5),"0.5":(-5,5),"1":(-5,5),"1.5":(-5,5)},
        verbose=2,
        random_state=1,
        allow_duplicate_points=True
    )
    import datetime

    dt = datetime.datetime.now()
    # datetime型  → 文字列型
    now_s = dt.strftime("%Y-%m-%d_%H-%M")
    logger = JSONLogger(path=f"./{now_s}-logs-bayesian10.json")
    bayesian_optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)


    setting.optSpectrum.frequency      = np.load('compensation_data/22030112_optFreq_LO_all_WS_range_spectre.npy')
    setting.optSpectrum.powerdensity   = np.load('compensation_data/22030112_optSpectrum_LO_all_WS_range_spectre.npy')
    setting.optSpectrum.phase          = np.zeros_like(setting.optSpectrum.frequency)

def loop_phase_bayesian10():
    # bayesian09のものを位相制御のみにしたもの
    # 中心 1545.4658 波長幅 0.5nm {-1.5,-1,-0.5,0,0.5,1,1.5}*1 + 1545.3nmの波長を制御します。 
    # 境界なめらか処理はなしです。まあなくてもいいんじゃない？
    # パワースペクトルの補償をするので、ポンプ光のスペクトルの中心通りにやります
    global state
    global bayesian_optimizer
    while True:
        try:
            idx_from = setting.speana_idx_from
            idx_to   = setting.speana_idx_to
            # 
            from bayes_opt import UtilityFunction
            utility = UtilityFunction(kind="ucb", kappa=0.5, xi=0.0)
            next_probe_point = bayesian_optimizer.suggest(utility)
            next_probe_point_trans = {float(k):v for k, v in next_probe_point.items()}
    #        next_probe_point[-3] = 0 # 境界なめらか処理
    #        next_probe_point[3] = 0 # 境界なめらか処理
            # phase generation(next_probe_point)
            band = Band()
            band.frequency          = setting.ws_Freqrange
            filterSpectrum          = gaussian_profile(
                        centerWavelength    =1545.4658,
                        bandWidth           =0.5, band=band,
                        dispersion          =-0.9
                    )  
            x = list(next_probe_point_trans.keys())
            x.sort()
            y = [next_probe_point_trans[i] for i in x]
            x = np.array([float(i) for i in x]) * 1.0 + 1545.3
            akima = interpolate.Akima1DInterpolator(x,y)
            filterSpectrum.phase = np.nan_to_num(akima(band.wavelength), 0)
            state.current_spectrum = filterSpectrum
            compensated_Spectrum    = compensation(
                setting.optSpectrum,
                targetSpectrum      =filterSpectrum,
                band                =band   
            )
            filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
            r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)

            close_pump_shutter()
            sleep(10)

            speana_handler = N9010BtransData(setting.n9010b_address)
            trace_shot = speana_handler.transfer(setting.hd_trace)
            shot_level = trace_shot[idx_from:idx_to].mean()

            open_pump_shutter()
            sleep(5)
            speana_handler = N9010BtransData(setting.n9010b_address)
            trace_shot = speana_handler.transfer(setting.hd_trace)
            sq_level = trace_shot[idx_from:idx_to].mean()


            bayesian_optimizer.register(
                params=next_probe_point,
                target= -(sq_level - shot_level), # 例えばベースはショットノイズレベルの平均パワー強度でいいと感じる。
                meta_data={'sq_level':sq_level, 'shot_level':shot_level}
            )
        except Exception: pass

def loop_speana():
    idx_from = setting.speana_idx_from
    idx_to   = setting.speana_idx_to
    global state
    # optimizing loop
    for filterSpectrum in param_gen_yield():
        ## new_LO_spectrum
        band = Band()
        band.frequency          = setting.ws_Freqrange
        compensated_Spectrum    = compensation(
                setting.optSpectrum,
                targetSpectrum      =filterSpectrum,
                band                =band   
            )
        total_power = calc_totalPower(
                                    setting.optSpectrum,
                                    compensated_Spectrum
                                )
        ## (power adjustment)
        #compensated_Spectrum = attenuation(compensated_Spectrum, total_power - setting.targetLOPower)
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)
        
        sleep(3)
        ## pump off
        servo_shutter(port=setting.shutter_port).close([setting.pumpShutterPort])
        sleep(1)
        ## speana_coms
        speana_handler = N9010BtransData(setting.n9010b_address)
        trace_shot = speana_handler.transfer(setting.hd_trace)
        shot_level = trace_shot[idx_from: idx_to].mean()

        ## pump on
        servo_shutter(port=setting.shutter_port).open([setting.pumpShutterPort])
        sleep(1)
        ## speana_com
        trace_pumped = speana_handler.transfer(setting.hd_trace)
        pumped_level = trace_pumped[idx_from: idx_to].mean()

        state.diff_history.append(pumped_level - shot_level)
        if pumped_level - shot_level > state.former_diff: param_unwind()
        else: state.former_diff = pumped_level - shot_level
        ## recording
        is_exists = os.path.exists(setting.param_csv)
        with open(setting.param_csv, 'a', newline = "") as f:
            headersCSV = ['LOpower', 'shotnoise', 'squeeze', 'diff']
            dict = {
                'LOpower'          : total_power,
                'shotnoise'        : shot_level,
                'squeeze'          : pumped_level,
                'diff'             : pumped_level - shot_level
            }
            dictwriter_object = DictWriter(f, fieldnames=headersCSV)
            if not is_exists: dictwriter_object.writeheader()
            dictwriter_object.writerow(dict)
        print(dict)
        


def loop():
    global state
    # optimizing loop
    for filterSpectrum in param_gen_yield():
        ## new_LO_spectrum
        band = Band()
        band.frequency          = setting.ws_Freqrange
        compensated_Spectrum    = compensation(
                setting.optSpectrum,
                targetSpectrum      =filterSpectrum,
                band                =band   
            )
        total_power = calc_totalPower(
                                    setting.optSpectrum,
                                    compensated_Spectrum
                                )
        ## (power adjustment)
        #compensated_Spectrum = attenuation(compensated_Spectrum, total_power - setting.targetLOPower)
        filterPort = np.ones(compensated_Spectrum.powerdensity.shape)
        r = uploadProfile(setting.ws_ip, compensated_Spectrum.frequency, -(compensated_Spectrum.powerdensity), compensated_Spectrum.phase, filterPort)
        
        sleep(3)
        ## pump off
        servo_shutter(port=setting.shutter_port).close([setting.pumpShutterPort])
        sleep(1)
        ## osc_com
        osc_handler = MSO58Wrapper(setting.mso58_address)
        osc_handler.push_single()
        t, w = None, None
        while True:
            try:
                t, w = osc_handler.transfer2byte(setting.homodyne_port)
                break
            except:
                sleep(1)
                continue
        ## quad
        quadratures = quad(w, setting.fastFramePulseNum, setting.quadrature_method)
        ## eval_shotnoise
        shot_var = quadratures.var()

        ## pump on
        servo_shutter(port=setting.shutter_port).open([setting.pumpShutterPort])
        sleep(1)
        ## osc_com
        osc_handler = MSO58Wrapper(setting.mso58_address)
        osc_handler.push_single()
        t, w = None, None
        while True:
            try:
                t, w = osc_handler.transfer2byte(setting.homodyne_port)
                break
            except:
                sleep(1)
                continue
        ## quad
        quadratures = quad(w, setting.fastFramePulseNum, setting.quadrature_method)
        ## eval_pumped
        pumped_var = quadratures.var()
        state.diff_history.append(10*np.log(pumped_var) - 10*np.log(shot_var))
        if 10*np.log(pumped_var) - 10*np.log(shot_var) > state.former_diff: param_unwind()
        else: state.former_diff = 10*np.log(pumped_var) - 10*np.log(shot_var)
        ## recording
        is_exists = os.path.exists(setting.param_csv)
        with open(setting.param_csv, 'a', newline = "") as f:
            headersCSV = ['LOpower', 'shotnoise', 'squeeze', 'diff']
            dict = {
                'LOpower'          : total_power,
                'shotnoise'        : 10*np.log(shot_var),
                'squeeze'          : 10*np.log(pumped_var),
                'diff'             : 10*np.log(pumped_var) - 10*np.log(shot_var)
            }
            dictwriter_object = DictWriter(f, fieldnames=headersCSV)
            if not is_exists: dictwriter_object.writeheader()
            dictwriter_object.writerow(dict)
        print(dict)
        

def init_loop():
    global state
    band = Band()
    band.frequency          = setting.ws_Freqrange
    filterSpectrum          = gaussian_profile(
                centerWavelength    =1545.3,
                bandWidth           =1, band=band,
                dispersion          =-0.9
            )
    state.former_spectrum = None
    state.current_spectrum = filterSpectrum

def spectrum_plot():
    global state
    powerdensity = state.current_spectrum.powerdensity
    wavelength = state.current_spectrum.wavelength
    phase = state.current_spectrum.phase
    plt.plot(wavelength, powerdensity)
    plt.ylim([-30,10])
    plt.show()

    plt.plot(wavelength, phase)
    plt.show()

In [ ]:
def read_osc():
    
    global t, w
    global now
    now = datetime.datetime.now()
    try:
        handler = MSO58transData(setting.mso58_address)
        t, w = handler.transfer2byte(1)
        print(f't.shape={t.shape} w.shape={w.shape}')
    except Exception as e:
        print('failed')
        print(e)
    

def save_osc():
    global comment
    filename_w = now.strftime('%Y%m%d_%H%M%S') + '_' + 'wdata_' + comment   + '.npy'
    #filename_w_tes = 'wdata_tes_' + comment + '_' + now.strftime('%Y%m%d_%H%M%S') + '.npy'
    filename_t = now.strftime('%Y%m%d_%H%M%S') + '_' +'tdata_' + comment +  '.npy'
    np.save(filename_w,w)
    #np.save(filename_w_tes,w_tes)
    np.save(filename_t,t)

def var_plot():
    global t, w
    plt.plot(np.var(w[:,:], axis = 0))
    plt.show()


def quadrature_plot():
    global pulseNum
    global hd_idx
    global w, t
    global comment
    global quadrature
    global quadratures
    global now
    global frames

    period = [int(i * w.shape[1]/pulseNum) for i in range(pulseNum)]

    volts = w.T

    varList = []

    frames = w.shape[0]

    fig = plt.figure(figsize = (40,5))
    quadratures = np.zeros((frames, pulseNum))

    ax1 = None
    ax2 = None
    for i in range(pulseNum):
        s = period[i]
        duration = int(w.shape[1]/pulseNum)
        ax1 = fig.add_subplot(2,max(pulseNum,10),i+1, sharey=ax1)
        ax1.plot(volts[s:(s+duration),1])
        ax1.set_title(f'{i}')
        #quadrature = volts[s:(s+duration),:].sum(axis=0)
        quadrature = volts[s+hd_idx,:]
        ax2 = fig.add_subplot(2,max(pulseNum,10),i+1 + max(pulseNum,10), sharey=ax2)
        ax2.scatter(range(w.shape[0]),quadrature,s = 0.01)
        ax2.set_title(f'{i}')
    plt.show()

    for j in range(frames):
        for i in range(pulseNum):
            s = period[i]
            #quadratures[j, i] = volts[s:s+duration, j].sum(axis=0)
            quadratures[j, i] = volts[s+hd_idx, j]

    varList = []

    for i in range(pulseNum):
        s = period[i]
        duration = int(w.shape[1]/pulseNum)
        #quadrature = volts[s:(s+duration),:].sum(axis=0)
        quadrature = volts[s+hd_idx,:]
        varList.append(quadrature.var())

    fig = plt.figure()
    plt.plot(varList)
    plt.gca().get_yaxis().get_major_formatter().set_useOffset(False)
    plt.show()

    plt.plot(10*np.log10(varList))
    plt.title('varList(dB)')
    plt.show

    fig2 = plt.figure()
    n, bins, patches = plt.hist(quadratures[:,1],bins=100)
    plt.show()

def save_quadratures():
    global comment
    global quadrature
    filename_hd = now.strftime('%Y%m%d_%H%M%S') + 'hd_' + comment + '.txt'
    np.savetxt(filename_hd, quadratures)

def save_hist_img():
    global comment
    global quadratures
    fig2 = plt.figure()
    n, bins, patches = plt.hist(quadratures[:,1],bins=100)
    plt.show()
    fig2.savefig(now.strftime('%Y%m%d_%H%M%S') + 'hist_' + comment + '.png')


def quadrature_sweep_plot():
    global w, t
    global sweep_target_pulse
    frameLength = w.shape[1]
    frames      = w.shape[0]
    period = [int(i * frameLength/pulseNum) for i in range(pulseNum)]
    duration = int(frameLength/pulseNum)
    quadratures = np.zeros((frames, pulseNum))

    volts = np.zeros((frames,pulseNum,duration))
    for i in range(pulseNum):
        s = period[i]
        volts[:,i,:] = w[:,s:(s+duration)]


    fig, ax = plt.subplots(2, 10, sharey="row", figsize = (30,5))
    for i, delta_hd_idx in enumerate(range(-5,5)):
        
        quadratures = volts.reshape((-1,volts.shape[-1]))[:,hd_idx + delta_hd_idx].reshape(volts.shape[:-1])
        ax1 = ax[0,i]
        ax1.scatter(range(quadratures.shape[0]),quadratures[:,sweep_target_pulse],s = 0.01)
        ax1.set_title(f'{delta_hd_idx + hd_idx}')
        ax2 = ax[1,i]
        ax2.hist(quadratures[:,sweep_target_pulse], bins=100)
    plt.show()
    pass

def save_spectrum():
    global state
    global comment
    powerdensity = copy.deepcopy(state.current_spectrum.powerdensity)
    phase = copy.deepcopy(state.current_spectrum.phase)
    frequency = copy.deepcopy(state.current_spectrum.frequency)
    filename = 'spectrum-' + comment + '.npy'
    np.save(filename, np.vstack([frequency, powerdensity, phase]))

def diff_history_plot():
    global state
    plt.plot(state.diff_history)
    plt.show()


In [ ]:
def simple_ui():
    button_clear_output = widgets.Button(description='表示クリア')
    button_read_osc = widgets.Button(description='オシロ読込')
    button_save_osc = widgets.Button(description='オシロ保存')
    button_var_plot = widgets.Button(description='varプロット')
    button_quadrature_plot = widgets.Button(description='quadratureプロット')
    button_quadrature_sweep_plot = widgets.Button(description='hd_idxスイーププロット')
    button_save_quadratures = widgets.Button(description='quadrature保存')
    button_save_hist_img = widgets.Button(description='hist保存')

    button_param_gen = widgets.Button(description='param_gen')
    button_spectrum_plot = widgets.Button(description='spectrumプロット')
    button_param_unwind = widgets.Button(description='param_unwind')
    button_set_param = widgets.Button(description='set_param')
    button_measure_diff_1push = widgets.Button(description='measure_diff_1push')
    button_init_loop = widgets.Button(description='初期化')
    button_loop = widgets.Button(description='loop')
    button_measure_diff_1push_speana = widgets.Button(description='measure_diff_1push_s')
    button_loop_speana = widgets.Button(description='loop_speana')
    button_measure_level_1push_speana = widgets.Button(description='measure_level_1push_s', layout={'width':'auto'})
    button_init_bayesian01 = widgets.Button(description='init_bayesian01', layout={'width':'auto'})
    button_loop_phase_bayesian01 = widgets.Button(description='loop_phase_bayesian01', layout={'width':'auto'})
    button_init_bayesian02 = widgets.Button(description='init_bayesian02', layout={'width':'auto'})
    button_loop_phase_bayesian02 = widgets.Button(description='loop_phase_bayesian02', layout={'width':'auto'})
    button_init_bayesian03 = widgets.Button(description='init_bayesian03', layout={'width':'auto'})
    button_loop_phase_bayesian03 = widgets.Button(description='loop_phase_bayesian03', layout={'width':'auto'})
    button_init_bayesian04 = widgets.Button(description='init_bayesian04', layout={'width':'auto'})
    button_loop_phase_bayesian04 = widgets.Button(description='loop_phase_bayesian04', layout={'width':'auto'})
    button_init_bayesian05 = widgets.Button(description='init_bayesian05', layout={'width':'auto'})
    button_loop_phase_bayesian05 = widgets.Button(description='loop_phase_bayesian05', layout={'width':'auto'})
    button_init_bayesian06 = widgets.Button(description='init_bayesian06', layout={'width':'auto'})
    button_loop_phase_bayesian06 = widgets.Button(description='loop_phase_bayesian06', layout={'width':'auto'})
    button_init_bayesian07 = widgets.Button(description='init_bayesian07', layout={'width':'auto'})
    button_loop_phase_bayesian07 = widgets.Button(description='loop_phase_bayesian07', layout={'width':'auto'})
    button_init_bayesian08 = widgets.Button(description='init_bayesian08', layout={'width':'auto'})
    button_loop_phase_bayesian08 = widgets.Button(description='loop_phase_bayesian08', layout={'width':'auto'})
    button_init_bayesian09 = widgets.Button(description='init_bayesian09', layout={'width':'auto'})
    button_loop_phase_bayesian09 = widgets.Button(description='loop_phase_bayesian09', layout={'width':'auto'})
    button_init_bayesian10 = widgets.Button(description='init_bayesian10', layout={'width':'auto'})
    button_loop_phase_bayesian10 = widgets.Button(description='loop_phase_bayesian10', layout={'width':'auto'})
    button_save_spectrum = widgets.Button(description='save_spectrum')
    button_diff_history_plot = widgets.Button(description='diff_history_plot')
    button_open_pump_shutter = widgets.Button(description='open_pump_shutter')
    button_close_pump_shutter = widgets.Button(description='close_pump_shutter')
    button_set_lo_EOM_high_voltage = widgets.Button(description='EOM_HIGH設定')



    filechooser = FileChooser('./')
    button_load_as_wdata_npy = widgets.Button(description='wdata.npyとして読込')
    button_load_as_tdata_npy = widgets.Button(description='tdata.npyとして読込')
    button_load_as_spectrum = widgets.Button(description='spectrum.npyとして読込')
    button_load_as_bayes_json = widgets.Button(description='bayes.jsonとして読込')
    button_input_field = widgets.Button(description='変数反映')

    text_comment = widgets.Text(value='',placeholder='文字を入力',description='comment',disabled=False)
    int_pulseNum = widgets.IntText(value=5,description='pulseNum')
    int_hd_idx = widgets.IntText(value=5,description='hd_idx')
    int_sweep_target_pulse = widgets.IntText(value=0, description='sweep_target_pulse')
    float_lo_EOM_high_voltage = widgets.FloatText(value=2.0, description='lo_EOM_high_voltage')


    output = widgets.Output(layout={'border': '1px solid black'})
    def wrapped_func_factory(func):
        def new_func(ui_element):
            with output:
                print(f"exec func {func.__name__}")
                func()
                print(f"complete {func.__name__}")
        return new_func
    button_clear_output.on_click(lambda button: output.clear_output(wait=False))
    button_read_osc.on_click(wrapped_func_factory(read_osc))
    button_save_osc.on_click(wrapped_func_factory(save_osc))
    button_var_plot.on_click(wrapped_func_factory(var_plot))
    button_quadrature_plot.on_click(wrapped_func_factory(quadrature_plot))
    button_quadrature_sweep_plot.on_click(wrapped_func_factory(quadrature_sweep_plot))
    button_save_quadratures.on_click(wrapped_func_factory(save_quadratures))
    button_save_hist_img.on_click(wrapped_func_factory(save_hist_img))
    button_param_gen.on_click(wrapped_func_factory(param_gen))
    button_spectrum_plot.on_click(wrapped_func_factory(spectrum_plot))
    button_param_unwind.on_click(wrapped_func_factory(param_unwind))
    button_set_param.on_click(wrapped_func_factory(set_param))
    button_measure_diff_1push.on_click(wrapped_func_factory(measure_diff_1push))
    button_init_loop.on_click(wrapped_func_factory(init_loop))
    button_loop.on_click(wrapped_func_factory(loop))
    button_measure_diff_1push_speana.on_click(wrapped_func_factory(measure_diff_1push_speana))
    button_loop_speana.on_click(wrapped_func_factory(loop_speana))
    button_save_spectrum.on_click(wrapped_func_factory(save_spectrum))
    button_diff_history_plot.on_click(wrapped_func_factory(diff_history_plot))
    button_measure_level_1push_speana.on_click(wrapped_func_factory(measure_level_1push_speana)) 
    button_init_bayesian01.on_click(wrapped_func_factory(init_bayesian01))
    button_loop_phase_bayesian01.on_click(wrapped_func_factory(loop_phase_bayesian01))
    button_init_bayesian02.on_click(wrapped_func_factory(init_bayesian02))
    button_loop_phase_bayesian02.on_click(wrapped_func_factory(loop_phase_bayesian02))
    button_open_pump_shutter.on_click(wrapped_func_factory(open_pump_shutter))
    button_close_pump_shutter.on_click(wrapped_func_factory(close_pump_shutter))
    button_init_bayesian03.on_click(wrapped_func_factory(init_bayesian03))
    button_loop_phase_bayesian03.on_click(wrapped_func_factory(loop_phase_bayesian03))
    button_init_bayesian04.on_click(wrapped_func_factory(init_bayesian04))
    button_loop_phase_bayesian04.on_click(wrapped_func_factory(loop_phase_bayesian04))
    button_init_bayesian05.on_click(wrapped_func_factory(init_bayesian05))
    button_loop_phase_bayesian05.on_click(wrapped_func_factory(loop_phase_bayesian05))
    button_init_bayesian06.on_click(wrapped_func_factory(init_bayesian06))
    button_loop_phase_bayesian06.on_click(wrapped_func_factory(loop_phase_bayesian06))
    button_init_bayesian07.on_click(wrapped_func_factory(init_bayesian07))
    button_loop_phase_bayesian07.on_click(wrapped_func_factory(loop_phase_bayesian07))
    button_init_bayesian08.on_click(wrapped_func_factory(init_bayesian08))
    button_loop_phase_bayesian08.on_click(wrapped_func_factory(loop_phase_bayesian08))
    button_init_bayesian09.on_click(wrapped_func_factory(init_bayesian09))
    button_loop_phase_bayesian09.on_click(wrapped_func_factory(loop_phase_bayesian09))
    button_init_bayesian10.on_click(wrapped_func_factory(init_bayesian10))
    button_loop_phase_bayesian10.on_click(wrapped_func_factory(loop_phase_bayesian10))
    button_set_lo_EOM_high_voltage.on_click(wrapped_func_factory(set_lo_EOM_high_voltage))


    def load_npy_factory(variable_name):
        def load_npy():
            choosed_file_path = filechooser.selected
            global w, t, state, bayesian_optimizer
            if variable_name == 'w':
                w = np.load(choosed_file_path)
                print(f'w.shape={w.shape}')
            elif variable_name == 't':
                t = np.load(choosed_file_path)
                print(f't.shape={t.shape}')
            elif variable_name == 'spectrum':
                freq, powerdensity, phase = np.load(choosed_file_path)
                band = Band()
                band.frequency = freq
                spectrum = Spectrum()
                spectrum.powerdensity = powerdensity
                spectrum.phase = phase
                spectrum.band = band
                state.current_spectrum = spectrum
            elif variable_name == 'bayes.json':
                load_logs(bayesian_optimizer, logs=[choosed_file_path])
        return load_npy
        
    button_load_as_wdata_npy.on_click(wrapped_func_factory(load_npy_factory('w')))
    button_load_as_tdata_npy.on_click(wrapped_func_factory(load_npy_factory('t')))
    button_load_as_spectrum.on_click(wrapped_func_factory(load_npy_factory('spectrum')))
    button_load_as_bayes_json.on_click(wrapped_func_factory(load_npy_factory('bayes.json')))
    
    def load_input_field():
        global comment
        global pulseNum
        global hd_idx
        global sweep_target_pulse
        global lo_EOM_high_voltage
        comment = text_comment.value
        pulseNum = int_pulseNum.value
        setting.fastFramePulseNum = pulseNum
        hd_idx = int_hd_idx.value
        sweep_target_pulse = int_sweep_target_pulse.value
        lo_EOM_high_voltage = float_lo_EOM_high_voltage.value
    button_input_field.on_click(lambda button: load_input_field())
    
    display(
        widgets.VBox([
            widgets.HBox([button_clear_output,button_read_osc,button_save_osc,button_var_plot,button_quadrature_plot,button_quadrature_sweep_plot,button_save_quadratures,button_save_hist_img]),
            widgets.HBox([button_param_gen, button_spectrum_plot, button_param_unwind, button_set_param, button_init_loop, button_diff_history_plot, button_save_spectrum]),
            widgets.HBox([button_close_pump_shutter, button_open_pump_shutter]),
            widgets.HBox([widgets.Label('オシロス用'),button_measure_diff_1push, button_loop]),
            widgets.HBox([widgets.Label('スペアナ用'),button_measure_diff_1push_speana,button_loop_speana]),
            widgets.HBox([widgets.Label('位相ベイズ01'),button_measure_level_1push_speana,button_init_bayesian01,button_loop_phase_bayesian01]),
            widgets.HBox([widgets.Label('位相ベイズ02'),button_init_bayesian02,button_loop_phase_bayesian02]),
            widgets.HBox([widgets.Label('位相ベイズ03(シャッター制御あり01)'),button_init_bayesian03,button_loop_phase_bayesian03]),
            widgets.HBox([widgets.Label('位相ベイズ04'), button_init_bayesian04, button_loop_phase_bayesian04]),
            widgets.HBox([widgets.Label('位相ベイズ05'), button_init_bayesian05, button_loop_phase_bayesian05]),
            widgets.HBox([widgets.Label('位相ベイズ06(LO角度ベイズ)'), button_init_bayesian06, button_loop_phase_bayesian06]),
            widgets.HBox([widgets.Label('位相ベイズ07(LO角度走査)'), button_init_bayesian07, button_loop_phase_bayesian07]),
            widgets.HBox([widgets.Label('位相ベイズ08(シャッター制御あり05)'), button_init_bayesian08, button_loop_phase_bayesian08]),
            widgets.HBox([widgets.Label('位相ベイズ09(中心波長、BW、位相、シャッター)'), button_init_bayesian09, button_loop_phase_bayesian09]),
            widgets.HBox([widgets.Label('位相ベイズ10(位相、シャッター、補償)'), button_init_bayesian10, button_loop_phase_bayesian10]),
            widgets.HBox([filechooser, button_load_as_wdata_npy, button_load_as_tdata_npy,button_load_as_spectrum,button_load_as_bayes_json]),
            widgets.HBox([text_comment,int_pulseNum, int_hd_idx,int_sweep_target_pulse]),
            widgets.HBox([float_lo_EOM_high_voltage,button_input_field]),
            output
        ])
    )

In [ ]:
simple_ui()